In [1]:
!pip install mediapipe 

In [2]:
import mediapipe as mp
import cv2

import csv
import os
import numpy as np
import pickle
import pandas as pd

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [13]:
with open('karate_points_model_manual.pkl', 'rb') as f:
    model = pickle.load(f)

In [4]:
index=["color", "color_name", "hex", "R", "G", "B"]
csv = pd.read_csv('colors.csv', names=index, header=None)


In [5]:
index_blue=["color_name", "R", "G", "B"]
csv_blue = pd.read_csv('colors_blue.csv', delimiter=";", names=index_blue, header=None)

In [6]:
csv_blue.loc[2, "R"]

17

In [7]:
print(csv_blue)

       color_name    R    G    B
0           Black    0    0    0
1    Darkest Blue    8   22   39
2            Navy   17   46   81
3       Dark Blue   32   84  147
4            Blue   46  120  210
5      Light Blue  109  161  224
6    Lighter Blue  151  188  233
7   Lightest Blue  193  215  242
8            Grey  120  144  156
9      Light Grey  167  192  205
10   Lighter Grey  200  215  223
11  Lightest Grey  232  293  242


In [8]:
#  Could be either .all() or any()

# If we want to recognize only Blue then we need a different color dataset 
# If we want to recognize only White then we only need the sum of all r,g,b values to be non-less then 690

# Calculating the distance from the actual colors from the dataset and whichever has the lowest distance wins the title of the color

def recognize_color(r,g,b):
 
    minimum = 10000
    cname = ""
    for i in range(len(csv_blue)):
        d = abs(r - int(csv_blue.loc[i,"R"])) + abs(g - int(csv_blue.loc[i,"G"]))+ abs(b - int(csv_blue.loc[i,"B"]))
        if (d <= minimum).all():
            minimum = d
            cname = csv_blue.loc[i,"color_name"]
    return cname

In [38]:
holistic.process??

In [9]:
# A logic can be implemented here to check for a specific color for 1 frame in a specific area 
# if the color is the one then mediapipe is allowed otherwise skiped 

# Or it can be first checked in "Extracting some points from Shoulder" and here it can be used

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
       
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    
    #Need to check whether it gets the real colors from the video
    coords_revert = coords_shoulder[::-1]
    color = image[coords_revert]
#     color = image[123,903]
    r = color[2]
    g = color[1]
    b = color[0]
    
    
    color_str = recognize_color(r,g,b)

#     print(color_str)
    return image, results, color_str

In [10]:
# If draw_styled_landmarks is applied no need of this method

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections


In [11]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [12]:
karate_video = "France_Spain_2010_full.mp4"

In [14]:
cap = cv2.VideoCapture(karate_video)
 
coords_shoulder = tuple()
color = []
color_str = ""

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    
    #this depends on the output video that is going to be put | the video and the internal camera need to be with the same size
    cap.set(3, 1280)
    cap.set(4, 720)
    
    print(cap.get(3))
    print(cap.get(4))
    print(cap.get(cv2.CAP_PROP_FPS))
    
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results, color_str = mediapipe_detection(frame, holistic)
#         print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        #Extracting some points from the shoulders
        try:
            coords_shoulder = tuple(np.multiply(
                    np.array(
                        (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x, 
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y))
                , [1280,720]).astype(int))

            cv2.rectangle(image, (coords_shoulder[0], coords_shoulder[1]+5), (coords_shoulder[0]+10, coords_shoulder[1]+15), (0, 0, 255), -1)

    #         cv2.rectangle(image, (900,120), (906,126), (255,0,0), 1)

            cv2.putText(image, color_str, (10,120), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)

#         Make detections based on the color that is taken from the right shoulder point
            if color_str == "Black" or color_str == "Darkest Blue" or color_str == "Navy":
                # Extract Pose landmarks
                pose = results.pose_landmarks.landmark
                pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

                # Concate rows
                row = pose_row

                # Make Detections
                X = pd.DataFrame([row])
                body_language_class = model.predict(X)[0]
                body_language_prob = model.predict_proba(X)[0]
                print(body_language_class, body_language_prob)

                # Grab ear coords
                coords = tuple(np.multiply(
                                np.array(
                                    (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                     results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                            , [640,480]).astype(int))

                cv2.rectangle(image, 
                              (coords[0], coords[1]+5), 
                              (coords[0]+len(body_language_class)*20, coords[1]-30), 
                              (245, 117, 16), -1)
                cv2.putText(image, body_language_class, coords, 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Get status box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

                # Display Class
                cv2.putText(image, 'CLASS'
                            , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, body_language_class.split(' ')[0]
                            , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display Probability
                cv2.putText(image, 'PROB'
                            , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                            , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        except:
            pass
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1280.0
720.0
1280.0
720.0
25.0
Nothing [0.13 0.02 0.05 0.27 0.03 0.01 0.49]
1 Point [0.28 0.07 0.18 0.18 0.06 0.01 0.22]
Nothing [0.02 0.   0.01 0.   0.01 0.   0.96]
Nothing [0.02 0.   0.01 0.   0.01 0.   0.96]
Nothing [0.02 0.   0.01 0.   0.01 0.   0.96]
Nothing [0.02 0.   0.01 0.   0.01 0.   0.96]
Nothing [0.01 0.   0.01 0.   0.01 0.   0.97]
Nothing [0.01 0.   0.01 0.   0.   0.   0.98]
Nothing [0.01 0.   0.01 0.   0.   0.   0.98]
Nothing [0.01 0.   0.01 0.   0.   0.   0.98]
Nothing [0.01 0.   0.01 0.   0.   0.   0.98]
Nothing [0.01 0.   0.01 0.   0.   0.   0.98]
Nothing [0.01 0.   0.01 0.   0.   0.   0.98]
Nothing [0.01 0.   0.01 0.   0.   0.   0.98]
Nothing [0.01 0.   0.01 0.   0.   0.   0.98]
Nothing [0.01 0.   0.01 0.   0.   0.   0.98]
Nothing [0.01 0.   0.01 0.   0.   0.   0.98]
Nothing [0.   0.   0.02 0.   0.   0.   0.98]
Nothing [0.   0.   0.01 0.   0.   0.   0.99]
Nothing [0.   0.   0.01 0.   0.   0.   0.99]
Nothing [0.   0.   0.01 0.   0.   0.   0.99]
Nothing [0. 0. 0. 0. 0. 

Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0.01 0.   0.   0.   0.   0.   0.99]
Nothing [0.01 0.   0.   0.   0.   0.   0.99]
Nothing [0.02 0.   0.   0.   0.   0.   0.98]
Nothing [0.01 0.   0.   0.   0.   0.   0.99]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0. 0. 0. 0. 0. 0. 1.]
Nothing [0.   0.   0.   0.   0.16 0.   0.84]
Nothing [0.   0.   0.   0.   0.31 0.   0.69]
Nothing [0.   0.   0.03 0.   0.02 0.07 0.88]
Nothing [0.01 0.   0.   0.   0.   0.   0.99]
Nothing [0.01 0.   0.   0.   0.   0.   0.99]
Nothing [0.01 0.   0.   0.   0.   0. 

Nothing [0.2  0.01 0.13 0.15 0.1  0.14 0.27]
Nothing [0.21 0.02 0.15 0.15 0.09 0.15 0.23]
1 Point [0.45 0.   0.1  0.23 0.02 0.   0.2 ]
Nothing [0.22 0.01 0.12 0.08 0.09 0.12 0.36]
Nothing [0.07 0.   0.01 0.01 0.03 0.04 0.84]
Nothing [0.06 0.   0.01 0.01 0.03 0.03 0.86]
Nothing [0.05 0.   0.01 0.01 0.03 0.04 0.86]
Nothing [0.05 0.   0.01 0.01 0.03 0.04 0.86]
Nothing [0.05 0.   0.01 0.01 0.03 0.03 0.87]
Nothing [0.06 0.   0.01 0.01 0.03 0.03 0.86]
Nothing [0.05 0.   0.01 0.01 0.03 0.03 0.87]
Nothing [0.05 0.   0.01 0.01 0.03 0.03 0.87]
Nothing [0.04 0.   0.02 0.01 0.03 0.03 0.87]
Nothing [0.04 0.   0.02 0.01 0.03 0.03 0.87]
Nothing [0.04 0.   0.02 0.01 0.03 0.03 0.87]
Nothing [0.03 0.   0.02 0.01 0.04 0.02 0.88]
Nothing [0.03 0.   0.01 0.01 0.05 0.02 0.88]
Nothing [0.02 0.   0.01 0.02 0.04 0.02 0.89]
Nothing [0.02 0.   0.01 0.02 0.04 0.02 0.89]
Nothing [0.02 0.   0.01 0.02 0.03 0.02 0.9 ]
Nothing [0.02 0.   0.01 0.02 0.03 0.02 0.9 ]
Nothing [0.02 0.   0.01 0.02 0.03 0.02 0.9 ]
Nothing [0

IndexError: index 1280 is out of bounds for axis 1 with size 1280

In [15]:
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
coords_shoulder

In [ ]:
int(image[coords_shoulder[0], coords_shoulder[1]])

image.shape

In [37]:
image.shape

(720, 1280, 3)

In [45]:
coords_shoulder

(1166, 457)

In [47]:
coords_revert = coords_shoulder[::-1]

In [48]:
coords_revert

(457, 1166)

In [ ]:
minimum = 10000
cname = ""
R = 0
G = 0
B = 0
for i in range(len(csv)):
    d = abs(R - int(csv.loc[i,"R"])) + abs(G - int(csv.loc[i,"G"]))+ abs(B - int(csv.loc[i,"B"]))
    if(d <= minimum):
        minimum = d
        cname = csv.loc[i,"color_name"]

In [ ]:
import numpy as np
a = np.array([1, 2, 3])
b = a.any()
print(b)